In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import glob
import librosa
import numpy as np

In [2]:
class AudioDataset(Dataset):
    def __init__(self, path, transform=None, sample_rate=16000):
        self.path = path
        self.data_list = glob.glob(self.path + '/*.wav')
        
        self.transform = transform
        self.sr = sample_rate
        self.frame_length = 0.025 # win_length, 자연어 처리 분야에서 25ms 크기를 기본으로 하고 있음 (16000Hz -> 400)
        self.frame_stride = 0.0126 # hop_length, 일반적으로 10ms의 크기를 기본으로 하고 있음 (16000Hz -> 160)
        
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, idx):
        data_path = self.data_list[idx]
        data = self.normalize(self.Mel_S(data_path))
        data = np.expand_dims(data, axis=0)
        
        if self.transform is not None:
            data = self.transform(data)
        
        return data # (1, 40, 80)
    
    def Mel_S(self, wav_file):
        y, sr = librosa.load(wav_file, sr=self.sr)
    
        input_nfft = int(round(sr*self.frame_length))
        input_stride = int(round(sr*self.frame_stride))

        s = librosa.feature.melspectrogram(y=y, n_mels=40, n_fft=input_nfft, hop_length=input_stride)
#         print(f"Wav lenght : {len(y)/sr}, Mes_S shape : {np.shape(S)}")
        
        return s
    
    def normalize(self, s):
        s = (s - s.mean()) / s.std() # standardization
        s = (s - s.min()) / (s.max() - s.min()) # min max normalization
        
        return s

In [3]:
dataset = AudioDataset(path='data/원천데이터/after')

In [4]:
dataset.__len__()
dataset_indices = list(range(dataset.__len__()))
np.random.shuffle(dataset_indices)
val_split_index = int(np.floor(0.2 * dataset.__len__()))
train_idx, val_idx = dataset_indices[val_split_index:], dataset_indices[:val_split_index]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [5]:
train_loader = DataLoader(dataset=dataset, batch_size=100, sampler=train_sampler, shuffle=False)
val_loader = DataLoader(dataset=dataset, batch_size=100, sampler=val_sampler, shuffle=False)

In [6]:
class VAE(nn.Module):
    def __init__(self, input_channel=1, h_dim=128*5*10, z_dim=512):
        super(VAE, self).__init__()
        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten()
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            nn.Unflatten(1, (128, 5, 10)),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )
        
    def reparameterization(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return mu + std*eps # return z sample
    
    def bottleneck(self, h):
        mu, log_var = self.fc1(h), self.fc2(h)
        z = self.reparameterization(mu, log_var)
        return z, mu, log_var
    
    def encode(self, x):
        h = self.encoder(x)
        z, mu, log_var = self.bottleneck(h)
        return z, mu, log_var
    
    def decode(self, z):
        z = self.fc3(z)
        return self.decoder(z)
    
    def forward(self, x):
        z, mu, log_var = self.encode(x)
        recon = self.decode(z)
        return recon, mu, log_var

vae = VAE()
if torch.cuda.is_available():
    vae.cuda()

In [7]:
vae

VAE(
  (encoder): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
  )
  (fc1): Linear(in_features=6400, out_features=512, bias=True)
  (fc2): Linear(in_features=6400, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=6400, bias=True)
  (decoder): Sequential(
    (0): Unflatten(dim=1, unflattened_size=(128, 5, 10))
    (1): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (2): ReLU()
    (3): ConvTranspose2d(64

In [8]:
optimizer = optim.Adam(vae.parameters())

def loss_function(recon_x, x, mu, log_var):
    KLD = -0.5*torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
#     RMSE = torch.sqrt(F.mse_loss(recon_x, x))
    return KLD + BCE

In [9]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()

        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)

        loss.backward()
        train_loss += loss.item()
        optimizer.step()

        if batch_idx % 10 == 0:
            print('Train Epoch : {} [{}/{} ({:.0f}%)]\tLoss : {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)
            ))
    print('====> Epoch : {} Average loss : {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [10]:
def val():
    vae.eval()
    val_loss = 0
    with torch.no_grad():
        for data in val_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)

            val_loss += loss_function(recon, data, mu, log_var).item()

    val_loss /= len(val_loader.dataset)
    print('====> Validation set loss : {:.4f}'.format(val_loss))

In [11]:
for epoch in range(1, 101):
    train(epoch)
    val()

Train Epoch : 1 [0/3375 (0%)]	Loss : 2504.483281
Train Epoch : 1 [1000/3375 (37%)]	Loss : 647.175977
Train Epoch : 1 [2000/3375 (74%)]	Loss : 475.245625
====> Epoch : 1 Average loss : 657.9008
====> Validation set loss : 76.6534
Train Epoch : 2 [0/3375 (0%)]	Loss : 401.040234
Train Epoch : 2 [1000/3375 (37%)]	Loss : 378.189766
Train Epoch : 2 [2000/3375 (74%)]	Loss : 293.626367
====> Epoch : 2 Average loss : 281.8653
====> Validation set loss : 66.0030
Train Epoch : 3 [0/3375 (0%)]	Loss : 291.307578
Train Epoch : 3 [1000/3375 (37%)]	Loss : 345.697891
Train Epoch : 3 [2000/3375 (74%)]	Loss : 307.918047
====> Epoch : 3 Average loss : 259.3653
====> Validation set loss : 62.9675
Train Epoch : 4 [0/3375 (0%)]	Loss : 316.663379
Train Epoch : 4 [1000/3375 (37%)]	Loss : 293.081289
Train Epoch : 4 [2000/3375 (74%)]	Loss : 319.163613
====> Epoch : 4 Average loss : 251.4497
====> Validation set loss : 61.8803
Train Epoch : 5 [0/3375 (0%)]	Loss : 331.084492
Train Epoch : 5 [1000/3375 (37%)]	Loss 

====> Epoch : 36 Average loss : 213.1486
====> Validation set loss : 53.0986
Train Epoch : 37 [0/3375 (0%)]	Loss : 267.987734
Train Epoch : 37 [1000/3375 (37%)]	Loss : 256.030469
Train Epoch : 37 [2000/3375 (74%)]	Loss : 242.260684
====> Epoch : 37 Average loss : 212.5991
====> Validation set loss : 53.0336
Train Epoch : 38 [0/3375 (0%)]	Loss : 249.940449
Train Epoch : 38 [1000/3375 (37%)]	Loss : 299.283242
Train Epoch : 38 [2000/3375 (74%)]	Loss : 279.298301
====> Epoch : 38 Average loss : 212.4889
====> Validation set loss : 53.0268
Train Epoch : 39 [0/3375 (0%)]	Loss : 260.593340
Train Epoch : 39 [1000/3375 (37%)]	Loss : 269.014922
Train Epoch : 39 [2000/3375 (74%)]	Loss : 251.457031
====> Epoch : 39 Average loss : 212.1045
====> Validation set loss : 52.9475
Train Epoch : 40 [0/3375 (0%)]	Loss : 267.740156
Train Epoch : 40 [1000/3375 (37%)]	Loss : 279.814824
Train Epoch : 40 [2000/3375 (74%)]	Loss : 261.612969
====> Epoch : 40 Average loss : 211.9955
====> Validation set loss : 53.

Train Epoch : 72 [1000/3375 (37%)]	Loss : 279.280977
Train Epoch : 72 [2000/3375 (74%)]	Loss : 301.459512
====> Epoch : 72 Average loss : 208.6407
====> Validation set loss : 52.1840
Train Epoch : 73 [0/3375 (0%)]	Loss : 259.727441
Train Epoch : 73 [1000/3375 (37%)]	Loss : 268.820312
Train Epoch : 73 [2000/3375 (74%)]	Loss : 245.022773
====> Epoch : 73 Average loss : 208.6014
====> Validation set loss : 52.0833
Train Epoch : 74 [0/3375 (0%)]	Loss : 275.622715
Train Epoch : 74 [1000/3375 (37%)]	Loss : 255.388223
Train Epoch : 74 [2000/3375 (74%)]	Loss : 253.353516
====> Epoch : 74 Average loss : 208.4875
====> Validation set loss : 52.0858
Train Epoch : 75 [0/3375 (0%)]	Loss : 240.156504
Train Epoch : 75 [1000/3375 (37%)]	Loss : 277.724727
Train Epoch : 75 [2000/3375 (74%)]	Loss : 294.654512
====> Epoch : 75 Average loss : 208.4603
====> Validation set loss : 52.1224
Train Epoch : 76 [0/3375 (0%)]	Loss : 259.749648
Train Epoch : 76 [1000/3375 (37%)]	Loss : 239.090957
Train Epoch : 76 [2

In [12]:
for epoch in range(101, 201):
    train(epoch)
    val()

Train Epoch : 101 [0/3375 (0%)]	Loss : 263.147168
Train Epoch : 101 [1000/3375 (37%)]	Loss : 259.479941
Train Epoch : 101 [2000/3375 (74%)]	Loss : 263.092461
====> Epoch : 101 Average loss : 207.2832
====> Validation set loss : 51.9598
Train Epoch : 102 [0/3375 (0%)]	Loss : 281.534668
Train Epoch : 102 [1000/3375 (37%)]	Loss : 231.331094
Train Epoch : 102 [2000/3375 (74%)]	Loss : 278.979297
====> Epoch : 102 Average loss : 207.2567
====> Validation set loss : 51.8334
Train Epoch : 103 [0/3375 (0%)]	Loss : 262.263926
Train Epoch : 103 [1000/3375 (37%)]	Loss : 261.598105
Train Epoch : 103 [2000/3375 (74%)]	Loss : 248.703320
====> Epoch : 103 Average loss : 207.0853
====> Validation set loss : 51.8414
Train Epoch : 104 [0/3375 (0%)]	Loss : 251.239727
Train Epoch : 104 [1000/3375 (37%)]	Loss : 285.127148
Train Epoch : 104 [2000/3375 (74%)]	Loss : 250.327793
====> Epoch : 104 Average loss : 207.1214
====> Validation set loss : 51.8359
Train Epoch : 105 [0/3375 (0%)]	Loss : 266.998867
Train 

====> Validation set loss : 51.6470
Train Epoch : 136 [0/3375 (0%)]	Loss : 266.557129
Train Epoch : 136 [1000/3375 (37%)]	Loss : 270.247168
Train Epoch : 136 [2000/3375 (74%)]	Loss : 256.687969
====> Epoch : 136 Average loss : 206.0463
====> Validation set loss : 51.6463
Train Epoch : 137 [0/3375 (0%)]	Loss : 259.960254
Train Epoch : 137 [1000/3375 (37%)]	Loss : 257.433984
Train Epoch : 137 [2000/3375 (74%)]	Loss : 258.280937
====> Epoch : 137 Average loss : 206.1201
====> Validation set loss : 51.6518
Train Epoch : 138 [0/3375 (0%)]	Loss : 265.500273
Train Epoch : 138 [1000/3375 (37%)]	Loss : 257.783887
Train Epoch : 138 [2000/3375 (74%)]	Loss : 267.657734
====> Epoch : 138 Average loss : 206.0579
====> Validation set loss : 51.6352
Train Epoch : 139 [0/3375 (0%)]	Loss : 237.161934
Train Epoch : 139 [1000/3375 (37%)]	Loss : 252.645977
Train Epoch : 139 [2000/3375 (74%)]	Loss : 247.523750
====> Epoch : 139 Average loss : 205.9351
====> Validation set loss : 51.6017
Train Epoch : 140 [0

====> Epoch : 170 Average loss : 205.5862
====> Validation set loss : 51.6117
Train Epoch : 171 [0/3375 (0%)]	Loss : 297.416211
Train Epoch : 171 [1000/3375 (37%)]	Loss : 249.485078
Train Epoch : 171 [2000/3375 (74%)]	Loss : 240.172773
====> Epoch : 171 Average loss : 205.6623
====> Validation set loss : 51.5970
Train Epoch : 172 [0/3375 (0%)]	Loss : 249.877539
Train Epoch : 172 [1000/3375 (37%)]	Loss : 238.379355
Train Epoch : 172 [2000/3375 (74%)]	Loss : 277.259863
====> Epoch : 172 Average loss : 205.3506
====> Validation set loss : 51.6094
Train Epoch : 173 [0/3375 (0%)]	Loss : 231.824863
Train Epoch : 173 [1000/3375 (37%)]	Loss : 286.561660
Train Epoch : 173 [2000/3375 (74%)]	Loss : 253.099102
====> Epoch : 173 Average loss : 205.3952
====> Validation set loss : 51.5280
Train Epoch : 174 [0/3375 (0%)]	Loss : 272.739355
Train Epoch : 174 [1000/3375 (37%)]	Loss : 285.075371
Train Epoch : 174 [2000/3375 (74%)]	Loss : 231.200918
====> Epoch : 174 Average loss : 205.4828
====> Validati

In [13]:
torch.save(vae.state_dict(), 'model/vae_mel_200.pt')

In [14]:
torch.save(vae, 'model/all_vae_mel_200.pt')

In [15]:
test_model = torch.load('model/all_vae_mel_200.pt')

In [19]:
test_model2 = VAE()

In [20]:
test_model2.load_state_dict(torch.load('model/vae_mel_200.pt'))

<All keys matched successfully>